In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_ga.csv") #- original geocoded file
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head() #10912

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,LongName,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long
0,265952,265952,1168647203,4/15/20,405,PPP,NEIGHBORHOOD RESTAURANT PARTNERS FLORIDA LLC,3060 Peachtree Rd NW Ste 400,ATLANTA,GA,...,Limited-Service Restaurants,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608
1,265953,265953,1393637109,4/10/20,405,PPP,"ATLANTA RESTAURANT PARTNERS, LLC",1760 Peachtree Road NW,ATLANTA,GA,...,Full-Service Restaurants,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271
2,265954,265954,1814537102,4/10/20,405,PPP,"MCCALLA RAYMER LEIBERT PIERCE, LLC",1544 OLD ALABAMA RD,ROSWELL,GA,...,Offices of Lawyers,999 Other,5411 Legal Services,5411.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648
3,265955,265955,1847307110,4/10/20,405,PPP,"C2 EDUCATIONAL SYSTEMS, INC.",6465 East Johns Crossing Suite 100,DULUTH,GA,...,Exam Preparation and Tutoring,999 Other,6116 Other Schools and Instruction,6116.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601
4,265956,265956,2680678506,2/22/21,405,PPP,MANAAIR LLC,1745 Phoenix Blvd Ste 250,College Park,GA,...,Scheduled Passenger Air Transportation,999 Other,4811 Scheduled Air Transportation,4811.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan 

,LoanNumber,full_add,Lat,Long
3735,1151418400,"229, CANTON, GA",NaN,NaN


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #10904

,LoanNumber,full_add,Lat,Long
0,1168647203,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608
1,1393637109,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271
2,1814537102,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648
3,1847307110,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601
4,2680678506,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [12]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/GA_fips_scraped.csv") 
fips_check_scraped = fips_check_scraped.drop(['Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1'], axis=1)
fips_check_scraped.head()

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,1168647203,4/15/20,405,PPP,NEIGHBORHOOD RESTAURANT PARTNERS FLORIDA LLC,3060 Peachtree Rd NW Ste 400,ATLANTA,GA,30305.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,1.312101e+14
1,1,1393637109,4/10/20,405,PPP,"ATLANTA RESTAURANT PARTNERS, LLC",1760 Peachtree Road NW,ATLANTA,GA,30309.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,1.312101e+14
2,2,1814537102,4/10/20,405,PPP,"MCCALLA RAYMER LEIBERT PIERCE, LLC",1544 OLD ALABAMA RD,ROSWELL,GA,30076.0,...,999 Other,5411 Legal Services,5411.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,1.312101e+14
3,3,1847307110,4/10/20,405,PPP,"C2 EDUCATIONAL SYSTEMS, INC.",6465 East Johns Crossing Suite 100,DULUTH,GA,30097.0,...,999 Other,6116 Other Schools and Instruction,6116.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,1.312101e+14
4,4,2680678506,2/22/21,405,PPP,MANAAIR LLC,1745 Phoenix Blvd Ste 250,College Park,GA,30349.0,...,999 Other,4811 Scheduled Air Transportation,4811.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,1.306304e+14


In [13]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [14]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [15]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #25831

,LoanNumber,full_add,Lat,Long,FIPS_z
0,1168647203,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,1.312101e+14
1,1393637109,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,1.312101e+14
2,1814537102,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,1.312101e+14
3,1847307110,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,1.312101e+14
4,2680678506,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,1.306304e+14
...,...,...,...,...,...
25827,9788108002,"4171 Marietta Street 300B, Powder Springs, GA",33.858438,-84.676229,NaN
25828,9791658407,"1210 Brookstone Centre Pkwy, Columbus, GA",32.537870,-84.978463,NaN
25829,9855898507,"201 Preserve Ct, Ball Ground, GA",34.332043,-84.349625,NaN
25830,9870708407,"8014 Cumming Hwy Ste 403-320, Canton, GA",34.252484,-84.359336,NaN


In [16]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #7119

,LoanNumber,full_add,Lat,Long,FIPS_z
7604,5679327103,"3220 US Hwy 78, LOGANVILLE, GA",33.616298,-85.764106,NaN
10221,6065397004,"400 BETH PAGE RD, THOMASVILLE, GA",30.678854,-84.189131,NaN
12228,9795107701,"21505 Augusta Ave, Port Charlotte, GA",26.989472,-82.092202,NaN
15843,8389928402,"951 US Highway 82 W Ste B, Leesburg, GA",33.630298,-100.374577,NaN
16089,8809168603,"6551 Palmer Park Circle, Sarasota, GA",27.260071,-82.478745,NaN
...,...,...,...,...,...
25827,9788108002,"4171 Marietta Street 300B, Powder Springs, GA",33.858438,-84.676229,NaN
25828,9791658407,"1210 Brookstone Centre Pkwy, Columbus, GA",32.537870,-84.978463,NaN
25829,9855898507,"201 Preserve Ct, Ball Ground, GA",34.332043,-84.349625,NaN
25830,9870708407,"8014 Cumming Hwy Ste 403-320, Canton, GA",34.252484,-84.359336,NaN


<h4>FIPS Script

In [17]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [18]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('13')] #GA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [19]:
geo_id_list =geo_id 

Length Check

In [20]:
print(len(geo_id_list))

7119


Matching FIPS to DataFrame

In [21]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:4431] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null

In [22]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

<ipython-input-22-08e9c917fe2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
7604,5679327103,"3220 US Hwy 78, LOGANVILLE, GA",33.616298,-85.764106,[]
10221,6065397004,"400 BETH PAGE RD, THOMASVILLE, GA",30.678854,-84.189131,[]
12228,9795107701,"21505 Augusta Ave, Port Charlotte, GA",26.989472,-82.092202,[]
15843,8389928402,"951 US Highway 82 W Ste B, Leesburg, GA",33.630298,-100.374577,[]
16089,8809168603,"6551 Palmer Park Circle, Sarasota, GA",27.260071,-82.478745,[]
...,...,...,...,...,...
25827,9788108002,"4171 Marietta Street 300B, Powder Springs, GA",33.858438,-84.676229,[130670315133031]
25828,9791658407,"1210 Brookstone Centre Pkwy, Columbus, GA",32.537870,-84.978463,[132150103032020]
25829,9855898507,"201 Preserve Ct, Ball Ground, GA",34.332043,-84.349625,[130570901013036]
25830,9870708407,"8014 Cumming Hwy Ste 403-320, Canton, GA",34.252484,-84.359336,[130570905042006]


<b>If FIPS_z > 1 :

In [53]:
#fips_fill['FIPS_z'] = geo_id
fips_loc = fips_fill
#coordinates with more than one return:
test = fips_loc[fips_loc['FIPS_z'].str.len() > 1]
test[['FIPS_x', 'FIPS_y', 'FIPS_z']]  = test['FIPS_z'].apply(lambda x: pd.Series(str(x).split(",")))
test = test.drop(['FIPS_x','FIPS_z'], axis=1)
test["FIPS_z"] = test["FIPS_y"]
test['FIPS_z']= test['FIPS_z'].replace(']', '', regex=True)
test['FIPS_z']= test['FIPS_z'].replace(' ', '', regex=True)
test['FIPS_z']= test['FIPS_z'].replace("'", '', regex=True)

cols = ['LoanNumber', 'FIPS_z']
test = test[cols]
fill_merge = fips_fill.merge(test, how="left", on="LoanNumber")
fill_merge

,LoanNumber,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,9734267002,"5357 CHAPEL HILL RD, DOUGLASVILLE, GA",33.659474,-84.714413,[130970805141005],NaN
1,5464177000,"1515 East Jackson St, THOMASVILLE, GA",30.855647,-83.954828,[132759606012004],NaN
2,1119698501,"4555 Mansell Rd Ste 230, Alpharetta, GA",34.039491,-84.288052,[131210114282000],NaN
3,3852517706,"4341 INTERNATIONAL PARKWAY SUITE 110, ATLANTA, GA",33.635347,-84.397485,[130630403081013],NaN
4,5338757301,"2100 Powers Ferry Rd SESuite 370, Atlanta, GA",33.901069,-84.451151,[130670303454000],NaN
...,...,...,...,...,...,...
14281,9456757006,"82 The American Way, DAWSONVILLE, GA",34.368023,-84.048026,"[134994843232020, 130859702061005]",130859702061005
14282,6123978606,"4780 Bakers Ferry Rd SW Ste E, Atlanta, GA",33.746340,-84.542465,[131210078051021],NaN
14283,8672777306,"5287 Covington Hwy, Decatur, GA",33.731479,-84.198501,[130890232132000],NaN
14284,3207688508,"86 Mount Vernon Hwy NW, Sandy Springs, GA",33.923809,-84.384902,[131210102201007],NaN


In [54]:
fill_merge['FIPS_z_x'] = fill_merge['FIPS_z_x'].str[0]
fill_merge['FIPS_z'] = fill_merge['FIPS_z_y'].fillna(fill_merge['FIPS_z_x'])
fill_merge = fill_merge.drop(['FIPS_z_x'], axis=1)
fill_merge = fill_merge.drop(['FIPS_z_y'], axis=1)
fips_fill = fill_merge
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,9734267002,"5357 CHAPEL HILL RD, DOUGLASVILLE, GA",33.659474,-84.714413,130970805141005
1,5464177000,"1515 East Jackson St, THOMASVILLE, GA",30.855647,-83.954828,132759606012004
2,1119698501,"4555 Mansell Rd Ste 230, Alpharetta, GA",34.039491,-84.288052,131210114282000
3,3852517706,"4341 INTERNATIONAL PARKWAY SUITE 110, ATLANTA, GA",33.635347,-84.397485,130630403081013
4,5338757301,"2100 Powers Ferry Rd SESuite 370, Atlanta, GA",33.901069,-84.451151,130670303454000
...,...,...,...,...,...
14281,9456757006,"82 The American Way, DAWSONVILLE, GA",34.368023,-84.048026,130859702061005
14282,6123978606,"4780 Bakers Ferry Rd SW Ste E, Atlanta, GA",33.746340,-84.542465,131210078051021
14283,8672777306,"5287 Covington Hwy, Decatur, GA",33.731479,-84.198501,130890232132000
14284,3207688508,"86 Mount Vernon Hwy NW, Sandy Springs, GA",33.923809,-84.384902,131210102201007


Remove brackets

In [23]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

<ipython-input-23-e19831f223ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
7604,5679327103,"3220 US Hwy 78, LOGANVILLE, GA",33.616298,-85.764106,NaN
10221,6065397004,"400 BETH PAGE RD, THOMASVILLE, GA",30.678854,-84.189131,NaN
12228,9795107701,"21505 Augusta Ave, Port Charlotte, GA",26.989472,-82.092202,NaN
15843,8389928402,"951 US Highway 82 W Ste B, Leesburg, GA",33.630298,-100.374577,NaN
16089,8809168603,"6551 Palmer Park Circle, Sarasota, GA",27.260071,-82.478745,NaN
...,...,...,...,...,...
25827,9788108002,"4171 Marietta Street 300B, Powder Springs, GA",33.858438,-84.676229,130670315133031
25828,9791658407,"1210 Brookstone Centre Pkwy, Columbus, GA",32.537870,-84.978463,132150103032020
25829,9855898507,"201 Preserve Ct, Ball Ground, GA",34.332043,-84.349625,130570901013036
25830,9870708407,"8014 Cumming Hwy Ste 403-320, Canton, GA",34.252484,-84.359336,130570905042006


In [24]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #10

,LoanNumber,full_add,Lat,Long,FIPS_z
7604,5679327103,"3220 US Hwy 78, LOGANVILLE, GA",33.616298,-85.764106,NaN
10221,6065397004,"400 BETH PAGE RD, THOMASVILLE, GA",30.678854,-84.189131,NaN
12228,9795107701,"21505 Augusta Ave, Port Charlotte, GA",26.989472,-82.092202,NaN
15843,8389928402,"951 US Highway 82 W Ste B, Leesburg, GA",33.630298,-100.374577,NaN
16089,8809168603,"6551 Palmer Park Circle, Sarasota, GA",27.260071,-82.478745,NaN
19563,3528947105,"P O Box 587 1975 N MAIN ST, LA FAYETTE, GA",34.066495,-118.218462,NaN
20213,9526287700,"613 WEST ASHLEY ST, JACKSONVILLE, GA",30.332706,-81.664247,NaN
21035,1077037104,"1504 HWY 195, LEESBURG, GA",30.724764,-97.670598,NaN
24798,8123187107,"1130 ADA ST., STE B, GA",29.381239,-98.455152,NaN
24851,7425547000,"9940 HIGHWAY 82, WOODSTOCK, GA",33.351056,-93.368070,NaN


Parse for Merge

In [25]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
7604,5679327103,NaN
10221,6065397004,NaN
12228,9795107701,NaN
15843,8389928402,NaN
16089,8809168603,NaN
...,...,...
25827,9788108002,130670315133031
25828,9791658407,132150103032020
25829,9855898507,130570901013036
25830,9870708407,130570905042006


Read in base data

In [26]:
#if from geocode file
#fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_ga.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("data/state_data/geo/geo_fips/GA_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check #15796


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,265952,265952,1168647203,4/15/20,405,PPP,NEIGHBORHOOD RESTAURANT PARTNERS FLORIDA LLC,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,1.312101e+14
1,1,1,1,265953,265953,1393637109,4/10/20,405,PPP,"ATLANTA RESTAURANT PARTNERS, LLC",...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,1.312101e+14
2,2,2,2,265954,265954,1814537102,4/10/20,405,PPP,"MCCALLA RAYMER LEIBERT PIERCE, LLC",...,999 Other,5411 Legal Services,5411.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,1.312101e+14
3,3,3,3,265955,265955,1847307110,4/10/20,405,PPP,"C2 EDUCATIONAL SYSTEMS, INC.",...,999 Other,6116 Other Schools and Instruction,6116.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,1.312101e+14
4,4,4,4,265956,265956,2680678506,2/22/21,405,PPP,MANAAIR LLC,...,999 Other,4811 Scheduled Air Transportation,4811.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,1.306304e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25827,25827,25827,25827,291779,291779,9788108002,7/8/20,405,PPP,"ALL WELL HEALTHCARE OF GA, INC",...,999 Other,6211 Offices of Physicians,6211.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4171 Marietta Street 300B, Powder Springs, GA",33.858438,-84.676229,NaN
25828,25828,25828,25828,291780,291780,9791658407,2/17/21,405,PPS,LANE DERMATOLOGY AND DERMATOLOGIC SURGERY LLC,...,999 Other,6211 Offices of Physicians,6211.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1210 Brookstone Centre Pkwy, Columbus, GA",32.537870,-84.978463,NaN
25829,25829,25829,25829,291781,291781,9855898507,3/12/21,405,PPS,POOL TECH LLC,...,999 Other,8114 Personal and Household Goods Repair and M...,8114.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"201 Preserve Ct, Ball Ground, GA",34.332043,-84.349625,NaN
25830,25830,25830,25830,291782,291782,9870708407,2/18/21,405,PPS,MBB KENNESAW MARKETPLACE LLC,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8014 Cumming Hwy Ste 403-320, Canton, GA",34.252484,-84.359336,NaN


Merge with MainFrame

In [27]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,0,0,265952,265952,1168647203,4/15/20,405,PPP,NEIGHBORHOOD RESTAURANT PARTNERS FLORIDA LLC,...,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,1.312101e+14,NaN
1,1,1,1,265953,265953,1393637109,4/10/20,405,PPP,"ATLANTA RESTAURANT PARTNERS, LLC",...,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,1.312101e+14,NaN
2,2,2,2,265954,265954,1814537102,4/10/20,405,PPP,"MCCALLA RAYMER LEIBERT PIERCE, LLC",...,5411 Legal Services,5411.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,1.312101e+14,NaN
3,3,3,3,265955,265955,1847307110,4/10/20,405,PPP,"C2 EDUCATIONAL SYSTEMS, INC.",...,6116 Other Schools and Instruction,6116.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,1.312101e+14,NaN
4,4,4,4,265956,265956,2680678506,2/22/21,405,PPP,MANAAIR LLC,...,4811 Scheduled Air Transportation,4811.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,1.306304e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25827,25827,25827,25827,291779,291779,9788108002,7/8/20,405,PPP,"ALL WELL HEALTHCARE OF GA, INC",...,6211 Offices of Physicians,6211.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4171 Marietta Street 300B, Powder Springs, GA",33.858438,-84.676229,NaN,130670315133031
25828,25828,25828,25828,291780,291780,9791658407,2/17/21,405,PPS,LANE DERMATOLOGY AND DERMATOLOGIC SURGERY LLC,...,6211 Offices of Physicians,6211.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1210 Brookstone Centre Pkwy, Columbus, GA",32.537870,-84.978463,NaN,132150103032020
25829,25829,25829,25829,291781,291781,9855898507,3/12/21,405,PPS,POOL TECH LLC,...,8114 Personal and Household Goods Repair and M...,8114.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"201 Preserve Ct, Ball Ground, GA",34.332043,-84.349625,NaN,130570901013036
25830,25830,25830,25830,291782,291782,9870708407,2/18/21,405,PPS,MBB KENNESAW MARKETPLACE LLC,...,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8014 Cumming Hwy Ste 403-320, Canton, GA",34.252484,-84.359336,NaN,130570905042006


If FIPS Encoded

In [28]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,0,0,265952,265952,1168647203,4/15/20,405,PPP,NEIGHBORHOOD RESTAURANT PARTNERS FLORIDA LLC,...,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,1.312101e+14,NaN,1.3121e+14
1,1,1,1,265953,265953,1393637109,4/10/20,405,PPP,"ATLANTA RESTAURANT PARTNERS, LLC",...,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,1.312101e+14,NaN,1.3121e+14
2,2,2,2,265954,265954,1814537102,4/10/20,405,PPP,"MCCALLA RAYMER LEIBERT PIERCE, LLC",...,5411.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,1.312101e+14,NaN,1.3121e+14
3,3,3,3,265955,265955,1847307110,4/10/20,405,PPP,"C2 EDUCATIONAL SYSTEMS, INC.",...,6116.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,1.312101e+14,NaN,1.3121e+14
4,4,4,4,265956,265956,2680678506,2/22/21,405,PPP,MANAAIR LLC,...,4811.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,1.306304e+14,NaN,1.3063e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25827,25827,25827,25827,291779,291779,9788108002,7/8/20,405,PPP,"ALL WELL HEALTHCARE OF GA, INC",...,6211.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4171 Marietta Street 300B, Powder Springs, GA",33.858438,-84.676229,NaN,130670315133031,130670315133031
25828,25828,25828,25828,291780,291780,9791658407,2/17/21,405,PPS,LANE DERMATOLOGY AND DERMATOLOGIC SURGERY LLC,...,6211.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1210 Brookstone Centre Pkwy, Columbus, GA",32.537870,-84.978463,NaN,132150103032020,132150103032020
25829,25829,25829,25829,291781,291781,9855898507,3/12/21,405,PPS,POOL TECH LLC,...,8114.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"201 Preserve Ct, Ball Ground, GA",34.332043,-84.349625,NaN,130570901013036,130570901013036
25830,25830,25830,25830,291782,291782,9870708407,2/18/21,405,PPS,MBB KENNESAW MARKETPLACE LLC,...,7225.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8014 Cumming Hwy Ste 403-320, Canton, GA",34.252484,-84.359336,NaN,130570905042006,130570905042006


Drop Extra FIPS Columns

In [29]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,265952,265952,1168647203,4/15/20,405,PPP,NEIGHBORHOOD RESTAURANT PARTNERS FLORIDA LLC,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,1.3121e+14
1,1,1,1,265953,265953,1393637109,4/10/20,405,PPP,"ATLANTA RESTAURANT PARTNERS, LLC",...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,1.3121e+14
2,2,2,2,265954,265954,1814537102,4/10/20,405,PPP,"MCCALLA RAYMER LEIBERT PIERCE, LLC",...,999 Other,5411 Legal Services,5411.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,1.3121e+14
3,3,3,3,265955,265955,1847307110,4/10/20,405,PPP,"C2 EDUCATIONAL SYSTEMS, INC.",...,999 Other,6116 Other Schools and Instruction,6116.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,1.3121e+14
4,4,4,4,265956,265956,2680678506,2/22/21,405,PPP,MANAAIR LLC,...,999 Other,4811 Scheduled Air Transportation,4811.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,1.3063e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25827,25827,25827,25827,291779,291779,9788108002,7/8/20,405,PPP,"ALL WELL HEALTHCARE OF GA, INC",...,999 Other,6211 Offices of Physicians,6211.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4171 Marietta Street 300B, Powder Springs, GA",33.858438,-84.676229,130670315133031
25828,25828,25828,25828,291780,291780,9791658407,2/17/21,405,PPS,LANE DERMATOLOGY AND DERMATOLOGIC SURGERY LLC,...,999 Other,6211 Offices of Physicians,6211.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1210 Brookstone Centre Pkwy, Columbus, GA",32.537870,-84.978463,132150103032020
25829,25829,25829,25829,291781,291781,9855898507,3/12/21,405,PPS,POOL TECH LLC,...,999 Other,8114 Personal and Household Goods Repair and M...,8114.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"201 Preserve Ct, Ball Ground, GA",34.332043,-84.349625,130570901013036
25830,25830,25830,25830,291782,291782,9870708407,2/18/21,405,PPS,MBB KENNESAW MARKETPLACE LLC,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8014 Cumming Hwy Ste 403-320, Canton, GA",34.252484,-84.359336,130570905042006


In [30]:
fips_merge.loc[fips_merge["FIPS_z"].isnull()] #11

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
3735,3735,3735,3735,269687,269687,1151418400,2/1/21,405,PPS,PHILLIP SMITH CONTRACTING INC.,...,2362.0,GEORGIA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"229, CANTON, GA",NaN,NaN,NaN,NaN,NaN
7604,7604,7604,7604,273556,273556,5679327103,4/13/20,405,PPP,"TRIAD MECHANICAL SERVICE SPECIALIST, LLC",...,2382.0,GEORGIA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"3220 US Hwy 78, LOGANVILLE, GA",33.616298,-85.764106,NaN,NaN,NaN
10221,10221,10221,10221,276173,276173,6065397004,4/6/20,405,PPP,"FCMC, LLC",...,5416.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"400 BETH PAGE RD, THOMASVILLE, GA",30.678854,-84.189131,NaN,NaN,NaN
12228,12228,12228,12228,278180,278180,9795107701,5/1/20,455,PPP,ST. CHARLES BORROMEO CATHOLIC SCHOOL,...,8131.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"21505 Augusta Ave, Port Charlotte, GA",26.989472,-82.092202,NaN,NaN,NaN
15843,15843,15843,15843,281795,281795,8389928402,2/13/21,405,PPS,MOLLY ANNE LLC,...,5614.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"951 US Highway 82 W Ste B, Leesburg, GA",33.630298,-100.374577,NaN,NaN,NaN
16089,16089,16089,16089,282041,282041,8809168603,3/25/21,405,PPS,R.F. CARLSON COMPANY,...,4238.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"6551 Palmer Park Circle, Sarasota, GA",27.260071,-82.478745,NaN,NaN,NaN
19563,19563,19563,19563,285515,285515,3528947105,4/11/20,405,PPP,JACKSON CHEVROLET BUICK GMC INC,...,4411.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"P O Box 587 1975 N MAIN ST, LA FAYETTE, GA",34.066495,-118.218462,NaN,NaN,NaN
20213,20213,20213,20213,286165,286165,9526287700,5/1/20,491,PPP,CLARA WHITE MISSION INC,...,6242.0,NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"613 WEST ASHLEY ST, JACKSONVILLE, GA",30.332706,-81.664247,NaN,NaN,NaN
21035,21035,21035,21035,286987,286987,1077037104,4/9/20,405,PPP,"SHARPCO, INC",...,4242.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1504 HWY 195, LEESBURG, GA",30.724764,-97.670598,NaN,NaN,NaN
24798,24798,24798,24798,290750,290750,8123187107,4/15/20,405,PPP,PULSE MEDICAL INC.,...,3345.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1130 ADA ST., STE B, GA",29.381239,-98.455152,NaN,NaN,NaN


In [32]:
geo_GA_fips = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1", "Unnamed: 0.1.1.1.1"], axis=1)
geo_GA_fips 

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,1168647203,4/15/20,405,PPP,NEIGHBORHOOD RESTAURANT PARTNERS FLORIDA LLC,3060 Peachtree Rd NW Ste 400,ATLANTA,GA,30305.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,1.3121e+14
1,1,1393637109,4/10/20,405,PPP,"ATLANTA RESTAURANT PARTNERS, LLC",1760 Peachtree Road NW,ATLANTA,GA,30309.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,1.3121e+14
2,2,1814537102,4/10/20,405,PPP,"MCCALLA RAYMER LEIBERT PIERCE, LLC",1544 OLD ALABAMA RD,ROSWELL,GA,30076.0,...,999 Other,5411 Legal Services,5411.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,1.3121e+14
3,3,1847307110,4/10/20,405,PPP,"C2 EDUCATIONAL SYSTEMS, INC.",6465 East Johns Crossing Suite 100,DULUTH,GA,30097.0,...,999 Other,6116 Other Schools and Instruction,6116.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,1.3121e+14
4,4,2680678506,2/22/21,405,PPP,MANAAIR LLC,1745 Phoenix Blvd Ste 250,College Park,GA,30349.0,...,999 Other,4811 Scheduled Air Transportation,4811.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,1.3063e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25827,25827,9788108002,7/8/20,405,PPP,"ALL WELL HEALTHCARE OF GA, INC",4171 Marietta Street 300B,Powder Springs,GA,30127.0,...,999 Other,6211 Offices of Physicians,6211.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4171 Marietta Street 300B, Powder Springs, GA",33.858438,-84.676229,130670315133031
25828,25828,9791658407,2/17/21,405,PPS,LANE DERMATOLOGY AND DERMATOLOGIC SURGERY LLC,1210 Brookstone Centre Pkwy,Columbus,GA,31904.0,...,999 Other,6211 Offices of Physicians,6211.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1210 Brookstone Centre Pkwy, Columbus, GA",32.537870,-84.978463,132150103032020
25829,25829,9855898507,3/12/21,405,PPS,POOL TECH LLC,201 Preserve Ct,Ball Ground,GA,30107.0,...,999 Other,8114 Personal and Household Goods Repair and M...,8114.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"201 Preserve Ct, Ball Ground, GA",34.332043,-84.349625,130570901013036
25830,25830,9870708407,2/18/21,405,PPS,MBB KENNESAW MARKETPLACE LLC,8014 Cumming Hwy Ste 403-320,Canton,GA,30115.0,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8014 Cumming Hwy Ste 403-320, Canton, GA",34.252484,-84.359336,130570905042006


Convert to File

In [33]:
geo_GA_fips.to_csv("data/state_data/geo/geo_fips/GA_fips_scraped.csv") 

Review & Compare

In [34]:
geo_GA_fips = pd.read_csv("data/state_data/geo/geo_fips/GA_fips_scraped.csv") 
geo_GA_fips

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,1168647203,4/15/20,405,PPP,NEIGHBORHOOD RESTAURANT PARTNERS FLORIDA LLC,3060 Peachtree Rd NW Ste 400,ATLANTA,GA,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"3060 Peachtree Rd NW Ste 400, ATLANTA, GA",33.839487,-84.381608,1.312101e+14
1,1,1,1393637109,4/10/20,405,PPP,"ATLANTA RESTAURANT PARTNERS, LLC",1760 Peachtree Road NW,ATLANTA,GA,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"1760 Peachtree Road NW, ATLANTA, GA",33.802087,-84.393271,1.312101e+14
2,2,2,1814537102,4/10/20,405,PPP,"MCCALLA RAYMER LEIBERT PIERCE, LLC",1544 OLD ALABAMA RD,ROSWELL,GA,...,999 Other,5411 Legal Services,5411.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1544 OLD ALABAMA RD, ROSWELL, GA",34.022578,-84.316648,1.312101e+14
3,3,3,1847307110,4/10/20,405,PPP,"C2 EDUCATIONAL SYSTEMS, INC.",6465 East Johns Crossing Suite 100,DULUTH,GA,...,999 Other,6116 Other Schools and Instruction,6116.0,GEORGIA DISTRICT OFFICE,e $5-10 million,500 or more,"6465 East Johns Crossing Suite 100, DULUTH, GA",34.062463,-84.167601,1.312101e+14
4,4,4,2680678506,2/22/21,405,PPP,MANAAIR LLC,1745 Phoenix Blvd Ste 250,College Park,GA,...,999 Other,4811 Scheduled Air Transportation,4811.0,GEORGIA DISTRICT OFFICE,e $5-10 million,250 to 499,"1745 Phoenix Blvd Ste 250, College Park, GA",33.615665,-84.446243,1.306304e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25827,25827,25827,9788108002,7/8/20,405,PPP,"ALL WELL HEALTHCARE OF GA, INC",4171 Marietta Street 300B,Powder Springs,GA,...,999 Other,6211 Offices of Physicians,6211.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4171 Marietta Street 300B, Powder Springs, GA",33.858438,-84.676229,1.306703e+14
25828,25828,25828,9791658407,2/17/21,405,PPS,LANE DERMATOLOGY AND DERMATOLOGIC SURGERY LLC,1210 Brookstone Centre Pkwy,Columbus,GA,...,999 Other,6211 Offices of Physicians,6211.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1210 Brookstone Centre Pkwy, Columbus, GA",32.537870,-84.978463,1.321501e+14
25829,25829,25829,9855898507,3/12/21,405,PPS,POOL TECH LLC,201 Preserve Ct,Ball Ground,GA,...,999 Other,8114 Personal and Household Goods Repair and M...,8114.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"201 Preserve Ct, Ball Ground, GA",34.332043,-84.349625,1.305709e+14
25830,25830,25830,9870708407,2/18/21,405,PPS,MBB KENNESAW MARKETPLACE LLC,8014 Cumming Hwy Ste 403-320,Canton,GA,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8014 Cumming Hwy Ste 403-320, Canton, GA",34.252484,-84.359336,1.305709e+14


In [36]:
#fill in missing Lat, Long, FIPS
geo_GA_fips.loc[geo_GA_fips["FIPS_z"].isnull()] #11

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
3735,3735,3735,1151418400,2/1/21,405,PPS,PHILLIP SMITH CONTRACTING INC.,229,CANTON,GA,...,999 Other,2362 Nonresidential Building Construction,2362.0,GEORGIA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"229, CANTON, GA",NaN,NaN,NaN
7604,7604,7604,5679327103,4/13/20,405,PPP,"TRIAD MECHANICAL SERVICE SPECIALIST, LLC",3220 US Hwy 78,LOGANVILLE,GA,...,999 Other,2382 Building Equipment Contractors,2382.0,GEORGIA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"3220 US Hwy 78, LOGANVILLE, GA",33.616298,-85.764106,NaN
10221,10221,10221,6065397004,4/6/20,405,PPP,"FCMC, LLC",400 BETH PAGE RD,THOMASVILLE,GA,...,999 Other,"5416 Management, Scientific, and Technical Con...",5416.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"400 BETH PAGE RD, THOMASVILLE, GA",30.678854,-84.189131,NaN
12228,12228,12228,9795107701,5/1/20,455,PPP,ST. CHARLES BORROMEO CATHOLIC SCHOOL,21505 Augusta Ave,Port Charlotte,GA,...,999 Other,8131 Religious Organizations,8131.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"21505 Augusta Ave, Port Charlotte, GA",26.989473,-82.092202,NaN
15843,15843,15843,8389928402,2/13/21,405,PPS,MOLLY ANNE LLC,951 US Highway 82 W Ste B,Leesburg,GA,...,999 Other,5614 Business Support Services,5614.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"951 US Highway 82 W Ste B, Leesburg, GA",33.630298,-100.374577,NaN
16089,16089,16089,8809168603,3/25/21,405,PPS,R.F. CARLSON COMPANY,6551 Palmer Park Circle,Sarasota,GA,...,999 Other,"4238 Machinery, Equipment, and Supplies Mercha...",4238.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"6551 Palmer Park Circle, Sarasota, GA",27.260071,-82.478745,NaN
19563,19563,19563,3528947105,4/11/20,405,PPP,JACKSON CHEVROLET BUICK GMC INC,P O Box 587 1975 N MAIN ST,LA FAYETTE,GA,...,999 Other,4411 Automobile Dealers,4411.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"P O Box 587 1975 N MAIN ST, LA FAYETTE, GA",34.066495,-118.218462,NaN
20213,20213,20213,9526287700,5/1/20,491,PPP,CLARA WHITE MISSION INC,613 WEST ASHLEY ST,JACKSONVILLE,GA,...,999 Other,"6242 Community Food and Housing, and Emergency...",6242.0,NORTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"613 WEST ASHLEY ST, JACKSONVILLE, GA",30.332706,-81.664247,NaN
21035,21035,21035,1077037104,4/9/20,405,PPP,"SHARPCO, INC",1504 HWY 195,LEESBURG,GA,...,999 Other,4242 Drugs and Druggists' Sundries Merchant Wh...,4242.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1504 HWY 195, LEESBURG, GA",30.724764,-97.670598,NaN
24798,24798,24798,8123187107,4/15/20,405,PPP,PULSE MEDICAL INC.,1130 ADA ST.,STE B,GA,...,334 Computers and Electronic Products,"3345 Navigational, Measuring, Electromedical, ...",3345.0,GEORGIA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1130 ADA ST., STE B, GA",29.381239,-98.455152,NaN
